In [1]:
import symforce
symforce.set_epsilon_to_symbol() # set epsilon to avoid singularities
from symforce.values import Values
import numpy as np
import cv2 as cv
import symforce.symbolic as sf
from symforce.opt.factor import Factor
from symforce.opt.optimizer import Optimizer

In [2]:
# ground truth coefficients
ar = 1.0
br = 2.0
cr = 1.0

# initial estimates
ae = 2.0
be = -1.0
ce = 5.0

# number of data points
N = 100

w_sigma = 1.0 # noise sigma
inv_sigma = 1.0 / w_sigma

rng = np.random.default_rng()

x_data = np.linspace(0, 1, N)
y_data = np.exp(ar * x_data * x_data + br * x_data + cr) + rng.normal(0, w_sigma, N)

In [3]:
def error(abc: sf.V3, x: sf.Scalar, y: sf.Scalar) -> sf.V1:
    return sf.V1(y - sf.exp(abc[0]*x*x + abc[1]*x + abc[2]))

In [4]:
initial_values = Values(
    abc = sf.V3([ae, be, ce]),
    xdata = x_data.tolist(),
    ydata = y_data.tolist(),
    epsilon = sf.numeric_epsilon,
)

In [5]:
# Each factor is the error between actual and predicted
factors = []
for i in range(len(x_data)):
    factors.append(Factor(
        residual=error,
        keys=["abc", f"xdata[{i}]", f"ydata[{i}]"],
    ))

In [6]:
optimizer = Optimizer(
    factors=factors,
    optimized_keys=["abc"],
    debug_stats=True,
    params=Optimizer.Params(iterations=1000)
)
result = optimizer.optimize(initial_values)
result.optimized_values.get('abc')

[2023-05-31 16:42:32.637] [info] LM<sym::Optimize> [iter    0] lambda: 1.000e+00, error prev/linear/new: 1644855.468/0.000/193592.062, rel reduction: 0.88230
[2023-05-31 16:42:32.642] [info] LM<sym::Optimize> [iter    1] lambda: 2.500e-01, error prev/linear/new: 193592.062/0.000/18234.643, rel reduction: 0.90581
[2023-05-31 16:42:32.647] [info] LM<sym::Optimize> [iter    2] lambda: 6.250e-02, error prev/linear/new: 18234.643/0.000/1104.470, rel reduction: 0.93943
[2023-05-31 16:42:32.652] [info] LM<sym::Optimize> [iter    3] lambda: 1.562e-02, error prev/linear/new: 1104.470/0.000/83.973, rel reduction: 0.92397
[2023-05-31 16:42:32.658] [info] LM<sym::Optimize> [iter    4] lambda: 3.906e-03, error prev/linear/new: 83.973/0.000/47.501, rel reduction: 0.43433
[2023-05-31 16:42:32.662] [info] LM<sym::Optimize> [iter    5] lambda: 9.766e-04, error prev/linear/new: 47.501/0.000/47.030, rel reduction: 0.00992
[2023-05-31 16:42:32.666] [info] LM<sym::Optimize> [iter    6] lambda: 2.441e-04, e

array([0.8250929 , 2.23868688, 0.92787156])

In [7]:
a, b, c = result.optimized_values.get('abc')
y_cal = np.exp(a * x_data * x_data + b * x_data + c)
y_gt = np.exp(ar * x_data * x_data + br * x_data + cr)

In [8]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.plot(x_data, y_data, '.')
plt.plot(x_data, y_cal)
plt.plot(x_data, y_gt)
plt.show()

qt5ct: using qt5ct plugin
